# 1. PREPARACIÓN NOTEBOOK

## 1.1 Librerías

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import joblib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import neighbors
from ipywidgets import interact
import os
import sys
import sqlite3 as sql

## 1.2 Conexión y carga de datos

In [ ]:
#Google Drive
drive.mount('/content/drive')

#Conectar al repositorio
path = '/content/drive/My Drive/cod/A3_marketing'

sys.path.append(path) #Importar las funciones propias a través de import, porque incluye la carpeta del repositorio como uno de esos paquetes para que import busque funciones
os.chdir(path) #Subir y descargar archivos de la ruta del repositorio de trabajo

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Funciones
%run a_funciones.ipynb

In [ ]:
DATABASE_COPY_PATH = '/content/drive/My Drive/cod/A3_marketing/data/db_movies_c'
con, cur = conectar_bd(DATABASE_COPY_PATH)

Conexión exitosa a la base de datos (copia): /content/drive/My Drive/cod/A3_marketing/data/db_movies_c


In [ ]:
df_tabla_final = pd.read_csv('/content/drive/My Drive/cod/A3_marketing/salidas/tabla_final.csv')
df_tabla_final.to_sql('tabla_final', con, if_exists='replace', index=False)

1104

In [ ]:
# Asegurarse que la carpeta 'salidas' existe en Google Drive
output_path = '/content/drive/My Drive/cod/A3_marketing/salidas'  # Ajusta esta ruta a la ubicación de tu carpeta
os.makedirs(output_path, exist_ok=True)

# Cargar datos preprocesados desde Google Drive
df_scaled = joblib.load(f"{output_path}/df_scaled.joblib")

# Cargar ratings si no los tienes cargados
ratings_path = '/content/drive/My Drive/cod/A3_marketing/data/ratings.csv'  # Ajusta esta ruta a la ubicación de tu archivo ratings.csv
ratings = pd.read_csv(ratings_path)

# 2. MODELOS

## 2.1 Recomendación basada en popularidad

In [ ]:
# Consulta 1: Películas mejor calificadas con al menos 20 calificaciones
consulta1 = """
SELECT title, avg_rating, num_ratings
FROM tabla_final
WHERE num_ratings >= 20
ORDER BY avg_rating DESC
LIMIT 10
"""
top_calificadas = pd.read_sql(consulta1, con)

In [ ]:
# Consulta 2: Las 10 películas más vistas
consulta2 = """
SELECT title, num_ratings, avg_rating
FROM tabla_final
ORDER BY num_ratings DESC
LIMIT 10
"""
mas_vistas = pd.read_sql(consulta2, con)

In [ ]:
#Consulta 3: Mejores 10 películas por década
consulta3 = """
SELECT decade, title, avg_rating
FROM (
    SELECT decade, title, avg_rating,
           ROW_NUMBER() OVER (
               PARTITION BY decade ORDER BY avg_rating DESC
           ) AS rank
    FROM tabla_final
) AS sub
WHERE rank <= 10
ORDER BY decade DESC, avg_rating DESC
"""
top_por_decada = pd.read_sql(consulta3, con)

## 2.2 Recomendación basada en contenido

In [ ]:
#Modelo KNN según una sola película
# Primero, selecciona solo las columnas numéricas para el modelo KNN (sin incluir la columna 'title')
columnas_numericas = df_scaled.drop(columns=['title', 'movieId'])

# Modelo KNN
model_knn = neighbors.NearestNeighbors(n_neighbors=11, metric='euclidean')
model_knn.fit(columnas_numericas)

# Obtener las distancias y los índices de los vecinos más cercanos
_, idlist = model_knn.kneighbors(columnas_numericas)

# Función para recomendar películas
def MovieRecommender(movie_name):
    # Encontrar el índice de la película en el DataFrame usando 'title'
    movie_id = df_scaled[df_scaled['title'] == movie_name].index[0]

    # Obtener los títulos de las películas recomendadas (sin incluir la propia película)
    return [df_scaled.iloc[i].title for i in idlist[movie_id][1:]]

# Usar interact para interactuar con el modelo
from ipywidgets import interact
interact(MovieRecommender, movie_name=df_scaled['title'].tolist())

interactive(children=(Dropdown(description='movie_name', options=('Toy Story', 'Jumanji', 'Grumpier Old Men', …

<function __main__.MovieRecommender(movie_name)>

In [49]:
#Modelo KNN según la decada de la película
# Primero, selecciona solo las columnas numéricas para el modelo KNN (sin incluir la columna 'title')
columnas_numericas = df_scaled.drop(columns=['title', 'movieId', 'decade'])

# Modelo KNN
model_knn = neighbors.NearestNeighbors(n_neighbors=11, metric='euclidean')
model_knn.fit(columnas_numericas)

# Obtener las distancias y los índices de los vecinos más cercanos
_, idlist = model_knn.kneighbors(columnas_numericas)

# Función para recomendar películas basadas en la misma década
def MovieRecommenderPorDecada(movie_name):
    # Encontrar el índice de la película en el DataFrame usando 'title'
    movie_id = df_scaled[df_scaled['title'] == movie_name].index[0]

    # Obtener la década de la película seleccionada
    decade_selected = df_scaled.loc[movie_id, 'decade']

    # Filtrar las películas que están en la misma década
    peliculas_de_misma_decada = df_scaled[df_scaled['decade'] == decade_selected]

    # Recomendación solo de las películas de la misma década
    # Obtener los títulos de las películas recomendadas (sin incluir la propia película)
    recomendaciones = []
    for i in idlist[movie_id][1:]:
        if df_scaled.iloc[i]['decade'] == decade_selected:
            recomendaciones.append(df_scaled.iloc[i].title)

    return recomendaciones

# Usar interact para interactuar con el modelo
from ipywidgets import interact
interact(MovieRecommenderPorDecada, movie_name=df_scaled['title'].tolist())

interactive(children=(Dropdown(description='movie_name', options=('Toy Story', 'Jumanji', 'Grumpier Old Men', …

<function __main__.MovieRecommenderPorDecada(movie_name)>

## 2.3 Recomendación filtro colaborativo

In [ ]:
# Filtrado colaborativo basado en ítem: Este sistema recomienda películas que son similares a las que el usuario ya ha visto y calificado,
#basándose en las calificaciones de otros usuarios

#Crear matriz de usuario x película
movie_user_matrix = ratings.pivot_table(index='movieId', columns='userId', values='rating').fillna(0)

# Calcular similitud
item_similarity = cosine_similarity(movie_user_matrix)
item_similarity_df = pd.DataFrame(item_similarity, index=movie_user_matrix.index, columns=movie_user_matrix.index)

# Diccionario de títulos a movieId
pelicula_dict = dict(zip(df_scaled['title'], df_scaled['movieId']))

def recomendar_por_item(movie_id, n=10):
    if movie_id not in item_similarity_df.index:
        return f"movieId {movie_id} no encontrado."
    similares = item_similarity_df[movie_id].sort_values(ascending=False)[1:n+1]
    recomendaciones = df_scaled[df_scaled['movieId'].isin(similares.index)][['movieId', 'title']].drop_duplicates()
    recomendaciones['similitud'] = similares.values
    return recomendaciones

@interact
def ver_recomendaciones_contenido(titulo=list(pelicula_dict.keys())):
    movie_id = pelicula_dict[titulo]
    display(recomendar_por_item(movie_id))

interactive(children=(Dropdown(description='titulo', options=('Toy Story', 'Jumanji', 'Grumpier Old Men', 'Fat…